In [55]:
# TODO

In [1]:
import csv
import gzip
import os

In [1]:
# headers = ['Chr', 'Position', 'rsID', 'Ref', 'Alt', 'Consequence', 'Canonical',
#            'Gene_symbol', 'LoF', 'LoF_flag', 'LoF_filter',
#            'AC', 'AC_afr', 'AC_amr', 'AC_nfe', 'AC_asj', 'AC_sas', 'AC_eas', 'AC_mid', 'AC_fin',
#            'AN', 'AN_afr', 'AN_amr', 'AN_nfe', 'AN_asj', 'AN_sas', 'AN_eas', 'AN_mid', 'AN_fin',
#            'AF', 'AF_afr', 'AF_amr', 'AF_nfe', 'AF_asj', 'AF_sas', 'AF_eas', 'AF_mid', 'AF_fin']

In [2]:
vcf_columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']
vcf_columns_mapping = dict(zip(vcf_columns, range(len(vcf_columns))))

In [3]:
vep_info_names = 'Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|ALLELE_NUM|DISTANCE|STRAND|FLAGS|VARIANT_CLASS|SYMBOL_SOURCE|HGNC_ID|CANONICAL|MANE_SELECT|MANE_PLUS_CLINICAL|TSL|APPRIS|CCDS|ENSP|UNIPROT_ISOFORM|SOURCE|||DOMAINS|miRNA|HGVS_OFFSET|PUBMED|MOTIF_NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|TRANSCRIPTION_FACTORS|LoF|LoF_filter|LoF_flags|LoF_info'.split('|')

In [4]:
def parse_line(line):
    parsed_data = []

    if line[vcf_columns_mapping['CHROM']].startswith('chr'):
        if line[vcf_columns_mapping['FILTER']] == 'PASS':

            chrom = line[vcf_columns_mapping['CHROM']]
            position = line[vcf_columns_mapping['POS']]
            rs_id = line[vcf_columns_mapping['ID']]
            ref = line[vcf_columns_mapping['REF']]
            alt = line[vcf_columns_mapping['ALT']]
    
            population_data = {
                'AC': None, 'AC_afr': None, 'AC_amr': None, 'AC_nfe': None, 'AC_asj': None, 'AC_sas': None, 'AC_eas': None, 'AC_mid': None, 'AC_fin': None,
                'AN': None, 'AN_afr': None, 'AN_amr': None, 'AN_nfe': None, 'AN_asj': None, 'AN_sas': None, 'AN_eas': None, 'AN_mid': None, 'AN_fin': None,
                'AF': None, 'AF_afr': None, 'AF_amr': None, 'AF_nfe': None, 'AF_asj': None, 'AF_sas': None, 'AF_eas': None, 'AF_mid': None, 'AF_fin': None
            }
            
            vep_data = {
                'Consequence': None, 'IMPACT': None, 'SYMBOL': None, 'Feature': None, 'BIOTYPE': None, 'CANONICAL': None, 
                'SOURCE': None, 'HGVSc': None, 'HGVSp': None, 'cDNA_position': None, 'STRAND': None, 'LoF': None, 'LoF_filter': None, 'LoF_flags': None
            }
            
            # Популяционные данные
            info_index = vcf_columns_mapping['INFO']
            column_with_info = line[info_index].split(';')
            for element in column_with_info:
                # содержит ли элемент (поле) информацию о популяционных частотах, указанных в словаре
                for category in population_data:
                    if element.startswith(f'{category}='):
                        frequency_value = element.split('=')
                        population_data[category] = frequency_value[-1] 
            
            vep_info = column_with_info[-1].split(',')
            
            for variant_info in vep_info:
                variant_info = element.split('|')
                for column_name, column_index in vep_data.items():
                    if column_name in vep_info_names:
                        vep_info_index = vep_info_names.index(column_name)
                        vep_data[column_name] = variant_info[vep_info_index]

            
                # Формируем строку данных для добавления в итоговый список
                filtered_data = [
                    chrom, position, rs_id, ref, alt, population_data['AC'],
                    vep_data['Consequence'], vep_data['IMPACT'], vep_data['SYMBOL'], vep_data['Feature'], vep_data['BIOTYPE'],
                    vep_data['CANONICAL'], vep_data['SOURCE'], vep_data['HGVSc'], vep_data['HGVSp'], vep_data['cDNA_position'], vep_data['STRAND'], 
                    vep_data['LoF'], vep_data['LoF_filter'], vep_data['LoF_flags']
                ]
                parsed_data.append(filtered_data)

    return parsed_data


In [5]:
headers = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'AC', 'Consequence', 'IMPACT', 'SYMBOL', 
           'Feature', 'BIOTYPE', 'CANONICAL', 'SOURCE', 'HGVSc', 'HGVSp', 'cDNA_position', 'STRAND', 'LoF', 'LoF_filter', 'LoF_flags']

def get_parse_vcf(vcf_file, output_dir='', output_filename=''):

    # Если папка не указана, сохраняем в текущую директорию
    if output_dir == '':
        output_dir = os.getcwd()
    
    # Если имя выходного файла не задано, используем имя входного файла, но с расширением .tsv
    # Через basename получаем имя файла, а через splitext оставляем только название без расширения
    if output_filename == '':
        output_filename = os.path.splitext(os.path.basename(vcf_file))[0] + '.tsv'
    else:
        # Если не указано расширение файла (указано неверно), добавляем .tsv
        if not output_filename.endswith('.tsv'):
            output_filename += '.tsv'
    
    # Полный путь к выходному файлу
    output_file = os.path.join(output_dir, output_filename)
    
    # Запись шапки таблицы в выходной файл
    with open(output_file, 'w', newline='', encoding='utf-8') as table_file:
        writer = csv.writer(table_file, delimiter='\t')
        writer.writerow(headers)

    
    with gzip.open(vcf_file, 'rt') as input_file, open(output_file, 'a', newline='', encoding='utf-8') as output_file:
        vcf_reader = csv.reader(input_file, delimiter='\t')
        tsv_writer = csv.writer(output_file, delimiter='\t')
        
        for line in vcf_reader:
            parsed_data = parse_line(line)
            for data in parsed_data:
                tsv_writer.writerow(data)

Указываем input файл (или путь к нему), а также имя output (необязательно, задаётся по умолчанию по названию вцф):

In [6]:
get_parse_vcf('../raw_data/example_22chr.vcf.bgz')

In [ ]:
get_parse_vcf('../raw_data/gnomad.exomes.v4.0.sites.chr21.vcf.bgz', output_dir='data_dir', output_filename='gnomad.exomes.v4.0.sites.chr21_all_new.vcf')

In [ ]:
get_parse_vcf('../raw_data/gnomad.exomes.v4.0.sites.chr13.vcf.bgz', output_dir='data_dir', output_filename='gnomad.exomes.v4.0.sites.chr13_all.vcf')

In [8]:
print('All done')

All done


In [ ]:
# chromosome_files = ['gnomad.exomes.v4.0.sites.chr1.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr2.vcf.bgz', 
#                     'gnomad.exomes.v4.0.sites.chr3.vcf.bgz', 
#                     'gnomad.exomes.v4.0.sites.chr4.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr5.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr6.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr7.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr8.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr9.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr10.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr11.vcf.bgz',
#                     'gnomad.exomes.v4.0.sites.chr12.vcf.bgz']

# for file_name in chromosome_files:
#     file_path = os.path.join('../raw_data', file_name)
#     get_parse_vcf(file_path, output_dir='data_dir')